In [1]:
import cv2
import numpy as np
import os
import math
import random
import csv

import tkinter.messagebox
from PIL import Image, ImageTk
from tkinter import *
from PIL import Image
from tkinter import filedialog
import tkinter
from tkinter.constants import *

In [2]:
plate=""
image=""
plate_text=""

#colour
SCALAR_BLACK=(0,0,0)
SCALAR_WHITE=(255,255,255)
SCALAR_YELLOW=(0,255,255)
SCALAR_GREEN=(0,255,0)
SCALAR_TEAL=(72,100,50)
SCALAR_RED=(255,0,0)


In [3]:
class PossibleChar:

    def __init__(self,_contour):
        self.contour=_contour
        self.boundingRect=cv2.boundingRect(self.contour)
        [intX,intY,intWidth,intHeight]=self.boundingRect
        self.intBoundingRectX=intX
        self.intBoundingRectY=intY
        self.intBoundingRectWidth=intWidth
        self.intBoundingRectHeight=intHeight
        self.intBoundingRectArea=self.intBoundingRectWidth*self.intBoundingRectHeight
        self.intCenterX=(self.intBoundingRectX+self.intBoundingRectX+self.intBoundingRectWidth)/2
        self.intCenterY=(self.intBoundingRectY+self.intBoundingRectY+self.intBoundingRectHeight)/2
        self.fltDiagonalSize=math.sqrt((self.intBoundingRectWidth**2)+(self.intBoundingRectHeight**2))
        self.fltAspectRatio=float(self.intBoundingRectWidth)/float(self.intBoundingRectHeight)  

In [4]:
class PossiblePlate:
    def __init__(self):
        self.imgPlate=None
        self.imgGrayscale=None
        self.imgThresh=None
        self.rrLocationOfPlateInScene=None
        self.strChars=""
        

### Pre-process

In [5]:
GAUSSIAN_SMOOTH_FILTER_SIZE=(5,5)
ADAPTIVE_THRESH_BLOCK_SIZE=19
ADAPTIVE_THRESH_WEIGHT=9


def preprocess(imgOriginal):
    imgGrayscale = extractValue(imgOriginal)
    imgMaxContrastGrayscale = maximizeContrast(imgGrayscale)
    height, width = imgGrayscale.shape
    imgBlurred = cv2.GaussianBlur(imgMaxContrastGrayscale, GAUSSIAN_SMOOTH_FILTER_SIZE, 0)
    # Apply CLAHE for better contrast
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    imgBlurred = clahe.apply(imgBlurred)
    imgThresh = cv2.adaptiveThreshold(imgBlurred, 255.0, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, ADAPTIVE_THRESH_BLOCK_SIZE, ADAPTIVE_THRESH_WEIGHT)
    return imgGrayscale, imgThresh

def extractValue(imgOriginal):
    height,width,numChannels=imgOriginal.shape
    imgHSV=np.zeros((height,width,3),np.uint8)
    imgHSV=cv2.cvtColor(imgOriginal,cv2.COLOR_BGR2HSV)
    imgHue,imgSaturation,imgValue=cv2.split(imgHSV)
    return imgValue

def maximizeContrast(imgGrayscale):

    height,width=imgGrayscale.shape
    imgTopHat=np.zeros((height,width,1),np.uint8)
    imgBlackHat=np.zeros((height,width,1),np.uint8)
    structuringElement=cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    imgTopHat=cv2.morphologyEx(imgGrayscale,cv2.MORPH_TOPHAT,structuringElement)
    imgBlackHat=cv2.morphologyEx(imgGrayscale,cv2.MORPH_BLACKHAT,structuringElement)
    imgGrayscalePlusTopHat=cv2.add(imgGrayscale,imgTopHat)
    imgGrayscalePlusTopHatMinusBlackHat=cv2.subtract(imgGrayscalePlusTopHat,imgBlackHat)
    return imgGrayscalePlusTopHatMinusBlackHat


### Detect Char

In [6]:
kNearest = cv2.ml.KNearest_create()

MIN_PIXEL_WIDTH = 2
MIN_PIXEL_HEIGHT = 8

MIN_ASPECT_RATIO = 0.25
MAX_ASPECT_RATIO = 1.0

MIN_PIXEL_AREA = 80

MIN_DIAG_SIZE_MULTIPLE_AWAY = 0.3
MAX_DIAG_SIZE_MULTIPLE_AWAY = 5.0

MAX_CHANGE_IN_AREA = 0.5

MAX_CHANGE_IN_WIDTH = 0.8
MAX_CHANGE_IN_HEIGHT = 0.2

MAX_ANGLE_BETWEEN_CHARS = 12.0

MIN_NUMBER_OF_MATCHING_CHARS = 3

RESIZED_CHAR_IMAGE_WIDTH = 20
RESIZED_CHAR_IMAGE_HEIGHT = 30

MIN_CONTOUR_AREA = 100
      
def loadKNNDataAndTrainKNN():
    allContoursWithData = []
    validContoursWithData = []

    try:
        npaClassifications = np.loadtxt("classifications.txt", np.float32)
    except:
        print("error, unable to open classifications.txt, exiting program\n")
        os.system("pause")
        return False
    try:
        npaFlattenedImages = np.loadtxt("flattened_images.txt", np.float32)
    except:
        print("error, unable to open flattened_images.txt, exiting program\n")
        os.system("pause")
        return False
    npaClassifications = npaClassifications.reshape((npaClassifications.size, 1))
    kNearest.setDefaultK(1)
    kNearest.train(npaFlattenedImages, cv2.ml.ROW_SAMPLE, npaClassifications)
    return True

def detectCharsInPlates(listOfPossiblePlates):
    intPlateCounter = 0
    imgContours = None
    contours = []

    if len(listOfPossiblePlates) == 0:
        return listOfPossiblePlates

    for possiblePlate in listOfPossiblePlates:
        possiblePlate.imgGrayscale, possiblePlate.imgThresh =preprocess(possiblePlate.imgPlate)
        possiblePlate.imgThresh = cv2.resize(possiblePlate.imgThresh,(0,0),fx=1.6, fy=1.6)
        thresholdValue, possiblePlate.imgThresh = cv2.threshold(possiblePlate.imgThresh,0.0,255.0, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
        listOfPossibleCharsInPlate = findPossibleCharsInPlate(possiblePlate.imgGrayscale, possiblePlate.imgThresh)
        listOfListsOfMatchingCharsInPlate = findListOfListsOfMatchingChars(listOfPossibleCharsInPlate)
        if (len(listOfListsOfMatchingCharsInPlate)==0):
            possiblePlate.strChars = ""
            continue
        for i in range(0,len(listOfListsOfMatchingCharsInPlate)):
            listOfListsOfMatchingCharsInPlate[i].sort(key=lambda matchingChar: matchingChar.intCenterX)
            listOfListsOfMatchingCharsInPlate[i]=removeInnerOverlappingChars(listOfListsOfMatchingCharsInPlate[i])
        intLenOfLongestListOfChars=0
        intIndexOfLongestListOfChars=0
        for i in range(0, len(listOfListsOfMatchingCharsInPlate)):
            if len(listOfListsOfMatchingCharsInPlate[i])>intLenOfLongestListOfChars:
                intLenOfLongestListOfChars=len(listOfListsOfMatchingCharsInPlate[i])
                intIndexOfLongestListOfChars=i
        longestListOfMatchingCharsInPlate=listOfListsOfMatchingCharsInPlate[intIndexOfLongestListOfChars]
        possiblePlate.strChars=recognizeCharsInPlate(possiblePlate.imgThresh,longestListOfMatchingCharsInPlate)    
    return listOfPossiblePlates

def findPossibleCharsInPlate(imgGrayscale,imgThresh):
    listOfPossibleChars=[]
    contours=[]
    imgThreshCopy=imgThresh.copy()
    contours, npaHierarchy=cv2.findContours(imgThreshCopy, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        ob1=PossibleChar(contour)
        possibleChar=ob1
        if checkIfPossibleChar(possibleChar):
            listOfPossibleChars.append(possibleChar)
    return listOfPossibleChars

def checkIfPossibleChar(possibleChar):
    if (possibleChar.intBoundingRectArea>MIN_PIXEL_AREA and
        possibleChar.intBoundingRectWidth>MIN_PIXEL_WIDTH and possibleChar.intBoundingRectHeight>MIN_PIXEL_HEIGHT and
        MIN_ASPECT_RATIO<possibleChar.fltAspectRatio and possibleChar.fltAspectRatio<MAX_ASPECT_RATIO):
        return True
    else:
        return False

def findListOfListsOfMatchingChars(listOfPossibleChars):
    listOfListsOfMatchingChars=[]
    for possibleChar in listOfPossibleChars:
        listOfMatchingChars=findListOfMatchingChars(possibleChar,listOfPossibleChars)
        listOfMatchingChars.append(possibleChar)

        if len(listOfMatchingChars)<MIN_NUMBER_OF_MATCHING_CHARS:
            continue
        listOfListsOfMatchingChars.append(listOfMatchingChars)
        listOfPossibleCharsWithCurrentMatchesRemoved=[]
        listOfPossibleCharsWithCurrentMatchesRemoved=list(set(listOfPossibleChars)-set(listOfMatchingChars))
        recursiveListOfListsOfMatchingChars=findListOfListsOfMatchingChars(listOfPossibleCharsWithCurrentMatchesRemoved)
        for recursiveListOfMatchingChars in recursiveListOfListsOfMatchingChars:
            listOfListsOfMatchingChars.append(recursiveListOfMatchingChars)
        break
    return listOfListsOfMatchingChars

def findListOfMatchingChars(possibleChar, listOfChars):
    listOfMatchingChars=[]

    for possibleMatchingChar in listOfChars:
        if possibleMatchingChar==possibleChar:
            continue
        fltDistanceBetweenChars=distanceBetweenChars(possibleChar, possibleMatchingChar)
        fltAngleBetweenChars=angleBetweenChars(possibleChar, possibleMatchingChar)
        fltChangeInArea=float(abs(possibleMatchingChar.intBoundingRectArea-possibleChar.intBoundingRectArea))/float(possibleChar.intBoundingRectArea)
        fltChangeInWidth=float(abs(possibleMatchingChar.intBoundingRectWidth-possibleChar.intBoundingRectWidth))/float(possibleChar.intBoundingRectWidth)
        fltChangeInHeight=float(abs(possibleMatchingChar.intBoundingRectHeight-possibleChar.intBoundingRectHeight))/float(possibleChar.intBoundingRectHeight)

        if (fltDistanceBetweenChars<(possibleChar.fltDiagonalSize*MAX_DIAG_SIZE_MULTIPLE_AWAY) and
            fltAngleBetweenChars<MAX_ANGLE_BETWEEN_CHARS and
            fltChangeInArea<MAX_CHANGE_IN_AREA and
            fltChangeInWidth<MAX_CHANGE_IN_WIDTH and
            fltChangeInHeight<MAX_CHANGE_IN_HEIGHT):
            listOfMatchingChars.append(possibleMatchingChar)

    return listOfMatchingChars

def distanceBetweenChars(firstChar,secondChar):
    intX=abs(firstChar.intCenterX-secondChar.intCenterX)
    intY=abs(firstChar.intCenterY-secondChar.intCenterY)
    return math.sqrt((intX**2)+(intY**2))

def angleBetweenChars(firstChar,secondChar):
    fltAdj=float(abs(firstChar.intCenterX-secondChar.intCenterX))
    fltOpp=float(abs(firstChar.intCenterY-secondChar.intCenterY))
    if fltAdj!=0.0:
        fltAngleInRad=math.atan(fltOpp/fltAdj)
    else:
        fltAngleInRad=1.5708
    fltAngleInDeg=fltAngleInRad*(180.0/math.pi)
    return fltAngleInDeg

def removeInnerOverlappingChars(listOfMatchingChars):
    listOfMatchingCharsWithInnerCharRemoved=list(listOfMatchingChars)

    for currentChar in listOfMatchingChars:
        for otherChar in listOfMatchingChars:
            if currentChar!=otherChar:
                if distanceBetweenChars(currentChar,otherChar)<(currentChar.fltDiagonalSize*MIN_DIAG_SIZE_MULTIPLE_AWAY):
                    if currentChar.intBoundingRectArea<otherChar.intBoundingRectArea:
                        if currentChar in listOfMatchingCharsWithInnerCharRemoved:
                            listOfMatchingCharsWithInnerCharRemoved.remove(currentChar)
                    else:
                        if otherChar in listOfMatchingCharsWithInnerCharRemoved:
                            listOfMatchingCharsWithInnerCharRemoved.remove(otherChar)
    return listOfMatchingCharsWithInnerCharRemoved

def recognizeCharsInPlate(imgThresh, listOfMatchingChars):
    strChars = ""
    height, width = imgThresh.shape
    imgThreshColor = np.zeros((height, width, 3), np.uint8)
    listOfMatchingChars.sort(key=lambda matchingChar: matchingChar.intCenterX)
    cv2.cvtColor(imgThresh, cv2.COLOR_GRAY2BGR, imgThreshColor)
    for currentChar in listOfMatchingChars:
        pt1 = (currentChar.intBoundingRectX, currentChar.intBoundingRectY)
        pt2 = ((currentChar.intBoundingRectX + currentChar.intBoundingRectWidth), (currentChar.intBoundingRectY + currentChar.intBoundingRectHeight))
        cv2.rectangle(imgThreshColor, pt1, pt2, SCALAR_GREEN, 2)
        imgROI = imgThresh[currentChar.intBoundingRectY : currentChar.intBoundingRectY + currentChar.intBoundingRectHeight,
                           currentChar.intBoundingRectX : currentChar.intBoundingRectX + currentChar.intBoundingRectWidth]
        imgROIResized = cv2.resize(imgROI, (RESIZED_CHAR_IMAGE_WIDTH, RESIZED_CHAR_IMAGE_HEIGHT))
        cv2.imwrite(f"debug_char_{currentChar.intCenterX}.png", imgROIResized)  # Save for inspection
        npaROIResized = imgROIResized.reshape((1, RESIZED_CHAR_IMAGE_WIDTH * RESIZED_CHAR_IMAGE_HEIGHT))
        npaROIResized = np.float32(npaROIResized)
        retval, npaResults, neigh_resp, dists = kNearest.findNearest(npaROIResized, k=3)  # Increased k to 3
        strCurrentChar = str(chr(int(npaResults[0][0])))
        print(f"Predicted: {strCurrentChar}, Raw result: {npaResults[0][0]}")  # Debug output
        strChars = strChars + strCurrentChar
    return strChars


### Detect Plate

In [ ]:
PLATE_WIDTH_PADDING_FACTOR=1.3
PLATE_HEIGHT_PADDING_FACTOR=1.5

def detectPlatesInScene(imgOriginalScene):
    listOfPossiblePlates = []
    imgGrayscaleScene = cv2.cvtColor(imgOriginalScene, cv2.COLOR_BGR2GRAY)
    imgThreshScene = cv2.adaptiveThreshold(imgGrayscaleScene, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
    contours, _ = cv2.findContours(imgThreshScene, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    print(f"Detected {len(contours)} contours")  # Debug
    for contour in contours:
        possiblePlate = PossiblePlate()
        possiblePlate.contour = contour
        possiblePlate.boundingRect = cv2.boundingRect(contour)
        x, y, w, h = possiblePlate.boundingRect
        aspectRatio = float(w) / h
        if 3 <= aspectRatio <= 5 and w * h > 5000:  # Tune for Indian plates
            possiblePlate.rrLocationOfPlateInScene = cv2.minAreaRect(contour)
            # Crop the plate image
            possiblePlate.imgPlate = imgOriginalScene[y:y+h, x:x+w]
            listOfPossiblePlates.append(possiblePlate)
            print(f"Added plate at ({x}, {y}) with size ({w}, {h})")  # Debug
    return listOfPossiblePlates

def findPossibleCharsInScene(imgThresh):
    listOfPossibleChars=[]
    intCountOfPossibleChars=0
    imgThreshCopy=imgThresh.copy()    
    contours,npaHierarchy=cv2.findContours(imgThreshCopy,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
    height,width=imgThresh.shape
    imgContours=np.zeros((height,width,3),np.uint8)
    for i in range(0,len(contours)):
        possibleChar=PossibleChar(contours[i])
        if checkIfPossibleChar(possibleChar):
            intCountOfPossibleChars=intCountOfPossibleChars + 1
            listOfPossibleChars.append(possibleChar)
    return listOfPossibleChars

def extractPlate(imgOriginal,listOfMatchingChars):
    ob1=PossiblePlate()
    possiblePlate=ob1
    listOfMatchingChars.sort(key=lambda matchingChar: matchingChar.intCenterX)
    fltPlateCenterX=(listOfMatchingChars[0].intCenterX+listOfMatchingChars[len(listOfMatchingChars)- 1].intCenterX) / 2.0
    fltPlateCenterY=(listOfMatchingChars[0].intCenterY+listOfMatchingChars[len(listOfMatchingChars)- 1].intCenterY) / 2.0
    ptPlateCenter=fltPlateCenterX,fltPlateCenterY
    intPlateWidth=int((listOfMatchingChars[len(listOfMatchingChars)-1].intBoundingRectX+listOfMatchingChars[len(listOfMatchingChars)-1].intBoundingRectWidth-listOfMatchingChars[0].intBoundingRectX)*PLATE_WIDTH_PADDING_FACTOR)
    intTotalOfCharHeights=0
    for matchingChar in listOfMatchingChars:
        intTotalOfCharHeights=intTotalOfCharHeights+matchingChar.intBoundingRectHeight
    fltAverageCharHeight=intTotalOfCharHeights/len(listOfMatchingChars)
    intPlateHeight=int(fltAverageCharHeight*PLATE_HEIGHT_PADDING_FACTOR)
    fltOpposite=listOfMatchingChars[len(listOfMatchingChars)-1].intCenterY-listOfMatchingChars[0].intCenterY
    fltHypotenuse=distanceBetweenChars(listOfMatchingChars[0],listOfMatchingChars[len(listOfMatchingChars)-1])
    fltCorrectionAngleInRad=math.asin(fltOpposite/fltHypotenuse)
    fltCorrectionAngleInDeg=fltCorrectionAngleInRad*(180.0/math.pi)
    possiblePlate.rrLocationOfPlateInScene=(tuple(ptPlateCenter),(intPlateWidth,intPlateHeight),fltCorrectionAngleInDeg)
    rotationMatrix=cv2.getRotationMatrix2D(tuple(ptPlateCenter),fltCorrectionAngleInDeg,1.0)
    height,width,numChannels=imgOriginal.shape
    imgRotated=cv2.warpAffine(imgOriginal,rotationMatrix,(width,height))
    imgCropped=cv2.getRectSubPix(imgRotated,(intPlateWidth,intPlateHeight),tuple(ptPlateCenter))
    possiblePlate.imgPlate=imgCropped
    return possiblePlate


: 

### Main

In [ ]:
def select_image():    
    # grab a reference to the image panels
    global panelA, panelB, image1, plate, plate_text
    # open a file chooser dialog and allow the user to select an input image
    path = filedialog.askopenfilename()  # Fixed: Use filedialog
    # ensure a file path was selected
    if len(path) > 0:        
        blnKNNTrainingSuccessful = loadKNNDataAndTrainKNN()
        if blnKNNTrainingSuccessful == False:
            print("\nerror: KNN training was not successful\n")
            return
        path1 = path
        path1 = (path1.split('/'))
        k = ((path1[(len(path1)-1)]).split('.')[0]) + ".png"
        imgOriginalScene = cv2.imread(path)
    
        if imgOriginalScene is None:
            print("\nerror: image not read from file \n\n")
            input("Press Enter to continue...")  # Cross-platform pause
            return
        else:
            listOfPossiblePlates = detectPlatesInScene(imgOriginalScene)
            listOfPossiblePlates = detectCharsInPlates(listOfPossiblePlates)         
            flag = 0
            
            # Ensure dataset/crops directory exists
            os.makedirs('dataset/crops', exist_ok=True)
            
            try:
                with open("plate.csv", "r") as f2:
                    f1 = csv.reader(f2, delimiter=',')
                    l = []
                    for row in f1:                
                        l.append(row)    
                    for i in range(0, len(l)):
                        if l[i][2] == k:
                            text = l[i][3]; xmin = l[i][4]; ymin = l[i][5]; xmax = l[i][6]; ymax = l[i][7]
                            flag = 1
                            break        
                if flag == 0:
                    if len(listOfPossiblePlates) == 0:
                        print("\nno license plates were detected\n")
                        return  # Exit early to avoid errors
                    else:
                        # Sort and select best plate
                        listOfPossiblePlates.sort(key=lambda possiblePlate: len(possiblePlate.strChars), reverse=True)
                        licPlate = listOfPossiblePlates[0]                
                        # Save crop
                        cv2.imwrite('dataset/crops/' + k, licPlate.imgPlate)
                        gray = cv2.cvtColor(licPlate.imgPlate, cv2.COLOR_BGR2GRAY)
                        gray = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]                    
                        if len(licPlate.strChars) == 0:
                            print("\nno characters were detected\n\n")
                            return
                        if licPlate.rrLocationOfPlateInScene is None:  # Safety check
                            print("Error: rrLocationOfPlateInScene is not set for this plate.")
                            xmin, ymin, xmax, ymax = licPlate.boundingRect[0], licPlate.boundingRect[1], licPlate.boundingRect[0] + licPlate.boundingRect[2], licPlate.boundingRect[1] + licPlate.boundingRect[3]
                        else:
                            xmin, ymin, xmax, ymax = drawRectangleAroundPlate(imgOriginalScene, licPlate)                    
                        text = licPlate.strChars
                        
                        # Append to CSV - check if l is empty
                        next_id = 1 if len(l) == 0 else (int((l[len(l)-1])[0]) + 1)
                        next_date = "2025-09-25" if len(l) == 0 else (int((l[len(l)-1])[1]) + 1)  # Assuming date column; adjust if needed
                        with open("plate.csv", 'a') as f3, open("plate.csv", "r") as f_temp:  # Re-read to avoid lock
                            if os.path.getsize("plate.csv") == 0:
                                writer = csv.writer(f3, delimiter=',')
                                writer.writerow(['ID', 'Date', 'Filename', 'Text', 'xmin', 'ymin', 'xmax', 'ymax'])  # Header if empty
                            else:
                                writer = csv.writer(f3, delimiter=',')
                                writer.writerow([next_id, next_date, k, licPlate.strChars, int(xmin), int(ymin), int(xmax), int(ymax)])                   
                if flag == 1:
                    imgOriginalScene = cv2.rectangle(imgOriginalScene, (int(xmin), int(ymax)), (int(xmax), int(ymin)), (255, 0, 0), 2)                           
                plate_text = text
                test = cv2.imread('dataset/crops/' + k)               
                writeLicensePlateCharsOnImage1(imgOriginalScene, xmin, ymin, text, test)              
                # convert the images to PIL format...
                plate = cv2.resize(test, (200, 80))
                image1 = cv2.resize(imgOriginalScene, (390, 240))
                # OpenCV represents images in BGR order; however PIL represents images in RGB order, so we need to swap the channels
                plate = cv2.cvtColor(plate, cv2.COLOR_BGR2RGB)
                image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
                plate = Image.fromarray(plate)
                image1 = Image.fromarray(image1)
        
                # ...and then to ImageTk format
                plate = ImageTk.PhotoImage(plate)
                image1 = ImageTk.PhotoImage(image1)
                number(plate_text)
                # if the panels are None, initialize them
                if panelA is None or panelB is None:
                    panelA = Label(MainFrame, image=image1)
                    panelA.grid(row=4, column=0, sticky=W)
                    panelB = Label(MainFrame, image=plate)
                    panelB.grid(row=4, column=1, sticky=W)        
                # otherwise, update the image panels
                else:
                    # update the panels
                    panelA.configure(image=image1)
                    panelA.image = image1  # Fixed: Use image1
                    panelA.grid(row=4, column=0, sticky=W)
                    panelB.configure(image=plate)
                    panelB.image = plate  # Fixed: Consistent
                    panelB.grid(row=4, column=1, sticky=W)
            except IndexError:
                print("CSV file is empty or malformed. Starting fresh.")

def drawRectangleAroundPlate(imgOriginalScene, licPlate):
    if licPlate.rrLocationOfPlateInScene is None:
        print("Warning: rrLocationOfPlateInScene is None, using bounding rect.")
        return licPlate.boundingRect[0], licPlate.boundingRect[1], licPlate.boundingRect[0] + licPlate.boundingRect[2], licPlate.boundingRect[1] + licPlate.boundingRect[3]
    p2fRectPoints = cv2.boxPoints(licPlate.rrLocationOfPlateInScene)
    # Convert float points to integer tuples
    intPoints = [(int(p[0]), int(p[1])) for p in p2fRectPoints]
    cv2.line(imgOriginalScene, intPoints[0], intPoints[1], SCALAR_RED, 2)
    cv2.line(imgOriginalScene, intPoints[1], intPoints[2], SCALAR_RED, 2)
    cv2.line(imgOriginalScene, intPoints[2], intPoints[3], SCALAR_RED, 2)
    cv2.line(imgOriginalScene, intPoints[3], intPoints[0], SCALAR_RED, 2)
    # Compute bounding box coordinates
    xs = [point[0] for point in intPoints]
    ys = [point[1] for point in intPoints]
    return min(xs), min(ys), max(xs), max(ys)  # xmin, ymin, xmax, ymax

def writeLicensePlateCharsOnImage1(imgOriginalScene, xmin, ymin, text, test):
    sceneHeight, sceneWidth, sceneNumChannels = imgOriginalScene.shape
    plateHeight, plateWidth, plateNumChannels = test.shape
    intFontFace = cv2.FONT_HERSHEY_SIMPLEX
    fltFontScale = 1
    intFontThickness = int(round(fltFontScale * 1.5))
    cv2.putText(imgOriginalScene, text, (int(xmin) - 21, int(ymin)), intFontFace, fltFontScale, SCALAR_WHITE, intFontThickness)

def number(num_detected):
    global text_widget  # Use global to update existing widget
    s = "Extracted Number is :\n"
    if 'text_widget' not in globals():
        text_widget = Text(MainFrame, font=('arial', 16, 'bold'), height=5, width=20)
        text_widget.grid(row=5, column=1, sticky=W)
    text_widget.delete(1.0, END)  # Clear previous text
    text_widget.insert(INSERT, (s + num_detected))

# GUI (rest of the code remains unchanged, assuming first cell with imports is run)
root = Tk()
panelA = None
panelB = None

root.title("GUI : Number Plate")

root.geometry("880x530")

root.configure(background='white')
Tops = Frame(root, bg='blue', pady=1, width=1850, height=90, relief="ridge")
Tops.grid(row=0, column=0)

Title_Label = Label(Tops, font=('Arial', 20, 'bold'), text="         Number plate detection using Python \t\t", pady=9, bg='white', fg='red', justify="center")
Title_Label.grid(row=0, column=0)
MainFrame = Frame(root, bg='white', pady=2, padx=2, width=1450, height=100, relief=RIDGE)
MainFrame.grid(row=1, column=0)

Label_1 = Label(MainFrame, font=('lato black', 17, 'bold'), text="", padx=2, pady=2, fg="black", bg="white")
Label_1.grid(row=0, column=0)

Label_9 = Button(MainFrame, font=('arial', 17, 'bold'), text="  Select Image\n Detect Plate ", padx=2, pady=2, bg="white", fg="black", command=select_image)
Label_9.grid(row=2, column=0, sticky=W)

Label_2 = Label(MainFrame, font=('arial', 10, 'bold'), text="\t\t    ", padx=2, pady=2, bg="white", fg="black")
Label_2.grid(row=3, column=0, sticky=W)

Label_3 = Label(MainFrame, font=('arial', 30, 'bold'), text="      \t\t\t", padx=2, pady=2, bg="white", fg="black")
Label_3.grid(row=4, column=0)

Label_3 = Label(MainFrame, font=('arial', 30, 'bold'), text="      \t\t\t", padx=2, pady=2, bg="white", fg="black")
Label_3.grid(row=5, column=0)

Label_3 = Label(MainFrame, font=('arial', 30, 'bold'), text="      \t\t\t", padx=2, pady=2, bg="white", fg="black")
Label_3.grid(row=6, column=0)

Label_3 = Label(MainFrame, font=('arial', 30, 'bold'), text="      \t\t\t", padx=2, pady=2, bg="white", fg="black")
Label_3.grid(row=7, column=0)

Label_3 = Label(MainFrame, font=('arial', 30, 'bold'), text="      \t\t\t", padx=2, pady=2, bg="white", fg="black")
Label_3.grid(row=8, column=0)

Label_3 = Label(MainFrame, font=('arial', 10, 'bold'), text="\t\t\t\t          ", padx=2, pady=2, bg="white", fg="black")
Label_3.grid(row=9, column=1)

root.mainloop()

Detected 782 contours

no license plates were detected

Detected 611 contours
Added plate at (113, 132) with size (162, 51)
Added plate at (56, 121) with size (144, 41)
Added plate at (55, 88) with size (204, 59)
Predicted: B, Raw result: 66.0
Predicted: V, Raw result: 86.0
Predicted: V, Raw result: 86.0
